# The JuMP ecosystem for mathematical optimization: MOI

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [1]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver